# Components

gdsfactory provides a generic customizable components library in `pp.components` 

## Basic shapes

### Rectangle

To create a simple rectangle, there are two functions:

``pp.components.rectangle()`` can create a basic rectangle:

In [ ]:
import pp
pp.components.rectangle(size = (4.5, 2), layer = (0, 0))

``pp.components.bbox()`` can also create a rectangle based on a bounding box. 
This is useful if you want to create a rectangle which exactly surrounds a piece of existing geometry. 
For example, if we have an arc geometry and we want to define a box around it, we can use ``pp.components.bbox()``: 

In [ ]:
D = pp.Component()
arc = D << pp.components.bend_circular(radius = 10, width = 0.5, angle = 90, layer = (1,0)).rotate(90)
# Draw a rectangle around the arc we created by using the arc's bounding box
rect = D << pp.components.bbox(bbox = arc.bbox, layer = (0,0))
D

### Cross

The ``pp.components.cross()`` function creates a cross structure:

In [ ]:
pp.components.cross(length = 10, width = 0.5, layer = (1,0))

### Ellipse

The ``pp.components.ellipse()`` function creates an ellipse by defining the major and minor radii:

In [ ]:
pp.components.ellipse(radii = (10,5), angle_resolution = 2.5, layer = (1, 0))

### Circle

The ``pp.components.circle()`` function creates a circle:

In [ ]:
pp.components.circle(radius = 10, angle_resolution = 2.5, layer = (1, 0))

### Ring

The ``pp.components.ring()`` function creates a ring.  The radius refers to the center radius of the ring structure (halfway between the inner and outer radius).

In [ ]:
pp.components.ring(radius = 5, width = 0.5, angle_resolution = 2.5, layer = (1, 0))

In [ ]:
pp.components.ring_single(width=0.5, gap=0.2, radius=10, length_x=4, length_y=2, layer=(1, 0))

In [ ]:
import pp
pp.components.ring_double(width=0.5, gap=0.2, radius=10, length_x=4, length_y=2, layer=(1,0))

In [ ]:
pp.components.ring_double(width=0.5, gap=0.2, radius=10, length_x=4, length_y=2, layer=(1,0), bend=pp.components.bend_circular)

### Bend circular

The ``pp.components.bend_circular()`` function creates an arc.  The radius refers to the center radius of the arc (halfway between the inner and outer radius). 

In [ ]:
pp.components.bend_circular(radius = 2.0, width = 0.5, angle = 90, npoints = 720, layer = (1,0))

### Bend euler

The ``pp.components.bend_euler()`` function creates an adiabatic bend in which the bend radius changes gradually. Euler bends have lower loss than circular bends.


In [ ]:
pp.components.bend_euler(radius = 2.0, width = 0.5, angle = 90, npoints = 720, layer = (1,0))

### Tapers

`pp.components.taper()`is defined by setting its length and its start and end length.  It has two ports, ``1`` and ``2``, on either end, allowing you to easily connect it to other structures.

In [ ]:
pp.components.taper(length = 10, width1 = 6, width2 = 4, port = None, layer = (1,0))

`pp.components.ramp()` is a structure is similar to `taper()` except it is asymmetric.  It also has two ports, ``1`` and ``2``, on either end.

In [ ]:
pp.components.ramp(length = 10, width1 = 4, width2 = 8, layer = (1,0))

### Common compound shapes

The `pp.components.L()` function creates a "L" shape with ports on either end named ``1`` and ``2``.

In [ ]:
pp.components.L(width = 7, size = (10,20) , layer = (1,0))

The `pp.components.C()` function creates a "C" shape with ports on either end named ``1`` and ``2``.

In [ ]:
pp.components.C(width = 7, size = (10,20) , layer = (1,0))

## Text

Gdsfactory has an implementation of the DEPLOF font with the majority of english ASCII characters represented (thanks to phidl)

In [ ]:
pp.components.text(text = 'Hello world!\nMultiline text\nLeft-justified', size = 10,
            justify = 'left', layer = (1,0))
# `justify` should be either 'left', 'center', or 'right'

## Grid / packer / align / distribute

### Grid


The ``pp.components.grid()`` function can take a list (or 2D array) of objects and arrange them along a grid. This is often useful for making parameter sweeps.   If the `separation` argument is true, grid is arranged such that the elements are guaranteed not to touch, with a `spacing` distance between them.  If `separation` is false, elements are spaced evenly along a grid. The `align_x`/`align_y` arguments specify intra-row/intra-column alignment.  The`edge_x`/`edge_y` arguments specify inter-row/inter-column alignment (unused if `separation = True`).

In [ ]:
import pp

components_list = []
for width1 in [1, 6, 9]:
    for width2 in [1, 2, 4, 8]:
        D = pp.components.taper(length = 10, width1 = width1, width2 = width2, layer = (1, 0))
        components_list.append(D)

c = pp.grid(components_list,
            spacing = (5,1),
            separation = True,
            shape = (3,4),
            align_x = 'x',
            align_y = 'y',
            edge_x = 'x',
            edge_y = 'ymax')
c

### Packer


The ``pp.pack()`` function packs geometries together into rectangular bins. If a ``max_size`` is specified, the function will create as many bins as is necessary to pack all the geometries and then return a list of the filled-bin Devices.

Here we generate several random shapes then pack them together automatically. We allow the bin to be as large as needed to fit all the Devices by specifying ``max_size = (None, None)``.  By setting ``aspect_ratio = (2,1)``, we specify the rectangular bin it tries to pack them into should be twice as wide as it is tall:

In [ ]:
import numpy as np
import pp

np.random.seed(5)
D_list = [pp.components.ellipse(radii = tuple(np.random.rand(2)*n+2)) for n in range(50)]
D_list += [pp.components.rectangle(size = tuple(np.random.rand(2)*n+2)) for n in range(50)]

D_packed_list = pp.pack(
        D_list,                 # Must be a list or tuple of Devices
        spacing = 1.25,         # Minimum distance between adjacent shapes
        aspect_ratio = (2,1),   # (width, height) ratio of the rectangular bin
        max_size = (None,None), # Limits the size into which the shapes will be packed
        density = 1.05,         # Values closer to 1 pack tighter but require more computation
        sort_by_area = True,    # Pre-sorts the shapes by area
        )
D = D_packed_list[0] # Only one bin was created, so we plot that
D

Say we need to pack many shapes into multiple 500x500 unit die. If we set ``max_size = (500,500)`` the shapes will be packed into as many 500x500 unit die as required to fit them all:

In [ ]:
np.random.seed(1)
D_list = [pp.components.ellipse(radii = tuple(np.random.rand(2)*n+2)) for n in range(120)]
D_list += [pp.components.rectangle(size = tuple(np.random.rand(2)*n+2)) for n in range(120)]

D_packed_list = pp.pack(
        D_list,                 # Must be a list or tuple of Devices
        spacing = 4,         # Minimum distance between adjacent shapes
        aspect_ratio = (1,1),   # Shape of the box
        max_size = (500,500),   # Limits the size into which the shapes will be packed
        density = 1.05,         # Values closer to 1 pack tighter but require more computation
        sort_by_area = True,    # Pre-sorts the shapes by area
        )

# Put all packed bins into a single device and spread them out with distribute()
F = pp.Component()
[F.add_ref(D) for D in D_packed_list]
F.distribute(elements = 'all', direction = 'x', spacing = 100, separation = True)
F

Note that the packing problem is an NP-complete problem, so ``pp.components.packer()`` may be slow if there are more than a few hundred Devices to pack (in that case, try pre-packing a few dozen at a time then packing the resulting bins). Requires the ``rectpack`` python package.

### Distribute


The ``distribute()`` function allows you to space out elements within a Device evenly in the x or y direction.  It is meant to duplicate the distribute functionality present in Inkscape / Adobe Illustrator:

![](images/inkscape_distribute.png)

Say we start out with a few random-sized rectangles we want to space out:

In [ ]:
c = pp.Component()
# Create different-sized rectangles and add them to D
[c.add_ref(pp.components.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
c

Oftentimes, we want to guarantee some distance between the objects.  By setting ``separation = True`` we move each object such that there is ``spacing`` distance between them:

In [ ]:
D = pp.Component()
# Create different-sized rectangles and add them to D
[D.add_ref(pp.components.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
# Distribute all the rectangles in D along the x-direction with a separation of 5
D.distribute(elements = 'all',   # either 'all' or a list of objects
             direction = 'x',    # 'x' or 'y'
             spacing = 5,
             separation = True)
D

Alternatively, we can spread them out on a fixed grid by setting ``separation = False``. Here we align the left edge (``edge = 'min'``) of each object along a grid spacing of 100:

In [ ]:
D = pp.Component()
[D.add_ref(pp.components.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 100, separation = False,
             edge = 'xmin') # edge must be either 'xmin' (left), 'xmax' (right), or 'x' (center)
D

The alignment can be done along the right edge as well by setting ``edge = 'max'``, or along the center by setting ``edge = 'center'`` like in the following:

In [ ]:
D = pp.Component()
[D.add_ref(pp.components.rectangle(size = [n*15+20,n*15+20]).move((n-10,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 100, separation = False,
             edge = 'x') # edge must be either 'xmin' (left), 'xmax' (right), or 'x' (center)
D

### Align


The ``align()`` function allows you to elements within a Device horizontally or vertically.  It is meant to duplicate the alignment functionality present in Inkscape / Adobe Illustrator:

![](images/inkscape_align.png)

Say we ``distribute()`` a few objects, but they're all misaligned:

In [ ]:
D = pp.Component()
# Create different-sized rectangles and add them to D then distribute them
[D.add_ref(pp.components.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 5, separation = True)
D

we can use the ``align()`` function to align their top edges (``alignment = 'ymax'):

In [ ]:
D = pp.Component()
# Create different-sized rectangles and add them to D then distribute them
[D.add_ref(pp.components.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 5, separation = True)

# Align top edges
D.align(elements = 'all', alignment = 'ymax')
D

or align their centers (``alignment = 'y'):

In [ ]:
D = pp.Component()
# Create different-sized rectangles and add them to D then distribute them
[D.add_ref(pp.components.rectangle(size = [n*15+20,n*15+20]).move((n,n*4))) for n in [0,2,3,1,2]]
D.distribute(elements = 'all', direction = 'x', spacing = 5, separation = True)

# Align top edges
D.align(elements = 'all', alignment = 'y')
D

other valid alignment options include ``'xmin', 'x', 'xmax', 'ymin', 'y', and 'ymax'``

## Boolean / outline / offset / invert
There are several common boolean-type operations available in the geometry library.  These include typical boolean operations (and/or/not/xor), offsetting (expanding/shrinking polygons), outlining, and inverting.

### Boolean


The ``pp.boolean()`` function can perform AND/OR/NOT/XOR operations, and will return a new geometry with the result of that operation.

Speedup note: The ``num_divisions`` argument can be used to divide up the geometry into multiple rectangular regions and process each region sequentially (which is more computationally efficient).  If you have a large geometry that takes a long time to process, try using ``num_divisions = [10,10]`` to optimize the operation.

In [ ]:
E = pp.components.ellipse(radii = (10,5), layer = (1,0))
R = pp.components.rectangle(size = [15,5], layer = (2,0)).movey(-1.5)
C = pp.boolean(A = E, B = R, operation = 'not', precision = 1e-6,
               num_divisions = [1,1], layer = 0)
# Other operations include 'and', 'or', 'xor', or equivalently 'A-B', 'B-A', 'A+B'

# Plot the originals and the result
D = pp.Component()
D.add_ref(E)
D.add_ref(R)
D.add_ref(C).movex(30)
D

### Offset

The ``pp.offset()`` function takes the polygons of the input geometry, combines them together, and expands/contracts them.  The function returns polygons on a single layer -- it does not respect layers.

Speedup note: The ``num_divisions`` argument can be used to divide up the geometry into multiple rectangular regions and process each region sequentially (which is more computationally efficient).  If you have a large geometry that takes a long time to process, try using ``num_divisions = [10,10]`` to optimize the operation.

In [ ]:
import pp

# Create `T`, an ellipse and rectangle which will be offset (expanded / contracted)
T = pp.Component('ellipse_and_rectangle')
e = T << pp.components.ellipse(radii = (10,5), layer = (1,0))
r = T << pp.components.rectangle(size = [15,5], layer = (2,0))
r.move([3,-2.5])

Texpanded = pp.offset(T, distance = 2, join_first = True, precision = 1e-6, 
        num_divisions = [1,1], layer = (0,0))
Texpanded.name = 'expanded'
Tshrink = pp.offset(T, distance = -1.5, join_first = True, precision = 1e-6, 
        num_divisions = [1,1], layer = (0,0))
Tshrink.name = 'shrink'

# Plot the original geometry, the expanded, and the shrunk versions
D = pp.Component('top')
t1 = D.add_ref(T)
t2 = D.add_ref(Texpanded)
t3 = D.add_ref(Tshrink)
D.distribute([t1,t2,t3], direction = 'x', spacing = 5)
D

### Outline

The ``pp.components.outline()`` function takes the polygons of the input geometry then performs an offset and "not" boolean operation to create an outline.  The function returns polygons on a single layer -- it does not respect layers.

Speedup note: The ``num_divisions`` argument can be used to divide up the geometry into multiple rectangular regions and process each region sequentially (which is more computationally efficient).  If you have a large geometry that takes a long time to process, try using ``num_divisions = [10,10]`` to optimize the operation.

In [ ]:
import phidl.geometry as pg

# Create a blank device and add two shapes
X = pp.Component()
X.add_ref(pp.components.cross(length = 25, width = 1, layer = (1,0)))
X.add_ref(pp.components.ellipse(radii = [10,5], layer = (2,0)))

O = pg.outline(X, distance = 1.5, precision = 1e-6, layer = (0, 0))

# Plot the original geometry and the result
D = pp.Component()
D.add_ref(X)
D.add_ref(O).movex(30)
D

The ``open_ports`` argument opens holes in the outlined geometry at each Port location. If not False, holes will be cut in the outline such that the Ports are not covered. If True, the holes will have the same width as the Ports. If a float, the holes will be widened by that value. If a float equal to the outline ``distance``, the outline will be flush with the port (useful positive-tone processes).

In [ ]:
# Create a geometry with ports
D = pp.components.L(width = 7, size = (10,20), layer = (1,0))

# Outline the geometry and open a hole at each port
N = pg.outline(D, distance = 5, open_ports = False) # No holes
O = pg.outline(D, distance = 5, open_ports = True)  # Hole is the same width as the port
P = pg.outline(D, distance = 5, open_ports = 2.9)  # Change the hole size by entering a float
Q = pg.outline(D, distance = 5, open_ports = 5)     # Creates flush opening (open_ports > distance)

# Plot the original geometry and the results
(D+N+O+P+Q).distribute(spacing = 10)
pp.plot([D,N,O,P,Q])

### Invert

The ``pg.invert()`` function creates an inverted version of the input geometry.  The function creates a rectangle around the geometry (with extra padding of distance ``border``), then subtract all polygons from all layers from that rectangle, resulting in an inverted version of the geometry.

Speedup note: The ``num_divisions`` argument can be used to divide up the geometry into multiple rectangular regions and process each region sequentially (which is more computationally efficient).  If you have a large geometry that takes a long time to process, try using ``num_divisions = [10,10]`` to optimize the operation.

In [ ]:
import phidl.geometry as pg

E = pp.components.ellipse(radii = (10,5))
D = pg.invert(E, border = 0.5, precision = 1e-6, layer = (0,0))
pp.plot(D)

### Union

The ``pg.union()`` function is a "join" function, and is functionally identical to the "OR" operation of ``pp.boolean()``.  The one difference is it's able to perform this function layer-wise, so each layer can be individually combined.

In [ ]:
import pp
import phidl.geometry as pg
from phidl import quickplot as plot

D = pp.Component()
D << pp.components.ellipse(layer = (0,0))
D << pp.components.ellipse(layer = (0,0)).rotate(15*1)
D << pp.components.ellipse(layer = (0,0)).rotate(15*2)
D << pp.components.ellipse(layer = (0,0)).rotate(15*3)
D << pp.components.ellipse(layer = (1,0)).rotate(15*4)
D << pp.components.ellipse(layer = (1,0)).rotate(15*5)

# We have two options to unioning - take all polygons, regardless of 
# layer, and join them together (in this case on layer 4) like so:
D_joined = pg.union(D, by_layer = False, layer = 4)

# Or we can perform the union operate by-layer
D_joined_by_layer = pg.union(D, by_layer = True)

# Space out shapes
D.add_ref(D_joined).movex(25)
D.add_ref(D_joined_by_layer).movex(50)
D

### XOR / diff

The ``pg.xor_diff()`` function can be used to compare two geometries and identify where they are different.  Specifically, it performs a layer-wise XOR operation.  If two geometries are identical, the result will be an empty Device.  If they are not identical, any areas not shared by the two geometries will remain.

In [ ]:
import phidl.geometry as pg

A = pp.Component()
A.add_ref(pp.components.ellipse(radii = [10,5], layer = (1,0)))
A.add_ref(pp.components.text('A')).move([3,0])
B = pp.Component()
B.add_ref(pp.components.ellipse(radii = [11,4], layer = (1,0)).movex(4))
B.add_ref(pp.components.text('B')).move([3.2,0])
X = pg.xor_diff(A = A, B = B, precision = 1e-6)

# Plot the original geometry and the result
# Upper left: A / Upper right: B
# Lower left: A and B / Lower right: A xor B "diff" comparison
D = pp.Component()
D.add_ref(A).move([-15,25])
D.add_ref(B).move([15,25])
D.add_ref(A).movex(-15)
D.add_ref(B).movex(-15)
D.add_ref(X).movex(15)
D

## Lithography structures

### Step-resolution

The `pp.components.litho_steps()` function creates lithographic test structure that is useful for measuring resolution of photoresist or electron-beam resists.  It provides both positive-tone and negative-tone resolution tests.

In [ ]:
D = pp.components.litho_steps(
        line_widths = [1,2,4,8,16],
        line_spacing = 10,
        height = 100,
        layer = (1,0)
        )
D

### Calipers (inter-layer alignment)

The `pp.components.litho_calipers()` function is used to detect offsets in multilayer fabrication.  It creates a two sets of notches on different layers.  When an fabrication error/offset occurs, it is easy to detect how much the offset is because both center-notches are no longer aligned.

In [ ]:
D = pp.components.litho_calipers(
        notch_size = [1,5],
        notch_spacing = 2,
        num_notches = 7,
        offset_per_notch = 0.1,
        row_spacing = 0,
        layer1 = (1,0),
        layer2 = (2,0))
D

## Paths / straights

See the **Path tutorial** for more details -- this is just an enumeration of the available built-in Path functions

### Circular arc

In [ ]:
P = pp.path.arc(radius = 10, angle = 135, npoints = 720)
pp.plot(P)

### Straight

In [ ]:
import pp
P = pp.path.straight(length = 5, npoints = 100)
pp.plot(P)

### Euler curve

Also known as a straight-to-bend, clothoid, racetrack, or track transition, this Path tapers adiabatically from straight to curved.  Often used to minimize losses in photonic straights.  If `p < 1.0`, will create a "partial euler" curve as described in Vogelbacher et. al. https://dx.doi.org/10.1364/oe.27.031394.  If the `use_eff` argument is false, `radius` corresponds to minimum radius of curvature of the bend.  If `use_eff`  is true, `radius` corresponds to the "effective" radius of the bend-- The curve will be scaled such that the endpoints match an arc with parameters `radius` and `angle`.

In [ ]:
P = pp.path.euler(radius = 3, angle = 90, p = 1.0, use_eff = False, npoints = 720)
pp.plot(P)

In [ ]:
P

### Smooth path from waypoints

In [ ]:
import numpy as np
import pp

points = np.array([(20,10), (40,10), (20,40), (50,40), (50,20), (70,20)])

P = pp.path.smooth(
    points = points, 
    radius = 2,
    bend_path_function = pp.path.euler,
    use_eff = False,
    )
pp.plot(P)

### Delay spiral

In [ ]:
pp.components.spiral()

## Importing GDS files

`pp.import_gds()` allows you to easily import external GDSII files.  It imports a single cell from the external GDS file and converts it into a gdsfactory component.

In [ ]:
D = pp.components.ellipse()
D.write_gds('myoutput.gds')
D = pp.import_gds(gdspath = 'myoutput.gds', cellname = None, flatten = False)
D

## LayerSet

The `LayerSet` class allows you to predefine a collection of layers and specify their properties including: gds layer/datatype, name, and color.  It also comes with a handy preview function called `pp.layers.preview_layerset()`

In [ ]:
import pp

lys = pp.layers.LayerSet()
lys.add_layer('p', color = 'lightblue', gds_layer = 1, gds_datatype = 0)
lys.add_layer('p+', color = 'blue', gds_layer = 2, gds_datatype = 0)
lys.add_layer('p++', color = 'darkblue', gds_layer = 3, gds_datatype = 0)
lys.add_layer('n', color = 'lightgreen', gds_layer = 4, gds_datatype = 0)
lys.add_layer('n+', color = 'green', gds_layer = 4, gds_datatype = 98)
lys.add_layer('n++', color = 'darkgreen', gds_layer = 5, gds_datatype = 99)
D = pp.layers.preview_layerset(lys, size = 100, spacing = 100)
D

## Useful contact pads / connectors

These functions are common shapes with ports, often used to make contact pads

In [ ]:
D = pp.components.compass(size = (4,2), layer = (1,0))
D

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.compass_multi(size = (4,2), ports = {'N':3,'S':4}, layer = (1,0))
plot(D) # quickplot the geometry

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.flagpole(size = (50,25), stub_size = (4,8), shape = 'p', taper_type = 'fillet', layer = (1,0))
# taper_type should be None, 'fillet', or 'straight'

plot(D) # quickplot the geometry

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.straight(size = (4,2), layer = (1,0))
plot(D) # quickplot the geometry

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.connector(midpoint = (0,0), width = 1, orientation = 0)
plot(D) # quickplot the geometry

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.tee(size = (8,4), stub_size = (1,2), taper_type = 'fillet', layer = (1,0))
# taper_type should be None, 'fillet', or 'straight'
plot(D)

## Chip / die template

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.basic_die(
              size = (10000, 5000), # Size of die
              street_width = 100,   # Width of corner marks for die-sawing
              street_length = 1000, # Length of corner marks for die-sawing
              die_name = 'chip99',  # Label text
              text_size = 500,      # Label text size
              text_location = 'SW', # Label text compass location e.g. 'S', 'SE', 'SW'
              layer = 0,
              draw_bbox = False,
              bbox_layer = 99,
              )
plot(D) # quickplot the geometry

## Optimal superconducting curves
The following structures are meant to reduce "current crowding" in superconducting thin-film structures (such as superconducting nanowires).  They are the result of conformal mapping equations derived in  Clem, J. & Berggren, K. "[Geometry-dependent critical currents in superconducting nanocircuits." Phys. Rev. B 84, 1–27 (2011).](http://dx.doi.org/10.1103/PhysRevB.84.174510)

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.optimal_hairpin(width = 0.2, pitch = 0.6, length = 10,
    turn_ratio = 4, num_pts = 50, layer = 0)
plot(D) # quickplot the geometry

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.optimal_step(start_width = 10, end_width = 22, num_pts = 50, width_tol = 1e-3,
                 anticrowding_factor = 1.2, symmetric = False, layer = 0)
plot(D) # quickplot the geometry

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.optimal_90deg(width = 100.0, num_pts = 15, length_adjust = 1, layer = 0)
plot(D) # quickplot the geometry

In [ ]:
import phidl.geometry as pg
from phidl import quickplot as plot

D = pg.snspd(wire_width = 0.2, wire_pitch = 0.6, size = (10,8),
        num_squares = None, turn_ratio = 4,
        terminals_same_side = False, layer = 0)
plot(D) # quickplot the geometry

In [ ]:
D = pg.snspd_expanded(wire_width = 0.3, wire_pitch = 0.6, size = (10,8),
           num_squares = None, connector_width = 1, connector_symmetric = False,
            turn_ratio = 4, terminals_same_side = False, layer = 0)
plot(D) # quickplot the geometry

## Copying and extracting geometry

In [ ]:
E = pp.Component()
E.add_ref(pp.components.ellipse(layer = (1,0)))
#X = pp.components.ellipse(layer = {0,1})
D = pg.extract(E, layers = [(1,0)])
plot(D) # quickplot the geometry

In [ ]:
import pp
X = pp.components.ellipse(layer=(2,0))
c = pp.copy(X)
c